In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [ ]:
batchSize = 4
imageSize = 256
#transforms.RandomCrop(imageSize),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#transforms.ToTensor()
#transforms.ToPILImage(),
data_transform = transforms.Compose([transforms.CenterCrop(imageSize),transforms.ToTensor()]
                                   )

In [ ]:
#data_transform

In [ ]:
#check for our GPU, otherwise, use a CPU

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else: 
    DEVICE = torch.device('cpu')
    

In [ ]:
USER = "peo5032"
DATA_FOLDER = '/home/'+USER+'/tensorflow/project'
dogs_dataset = datasets.ImageFolder(root=DATA_FOLDER,
                                           transform=data_transform)
data_loader = torch.utils.data.DataLoader(dogs_dataset,
                                             batch_size=batchSize, shuffle=True,
                                             num_workers=4)

In [ ]:
#reference = iter(data_loader)
#reference.next()

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02).to(device=DEVICE)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02).cuda()
        m.bias.data.fill_(0).cuda()


In [ ]:
class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 300, 10, 1, 0, bias = False),
            nn.BatchNorm2d(300),
            nn.ReLU(True),
            nn.ConvTranspose2d(300, 256, 10, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 10, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 256, 10, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 3, 10, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input.cuda())
        return output.cuda()
    
netG = G().cuda()
netG.apply(weights_init).cuda()

In [ ]:
class D(nn.Module):

    def __init__(self):
        #print("HELLO FROM CONSTRUCTOR OF D")
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 256, 10, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 128, 10, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 512, 10, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 256, 10, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 1, 10, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        
        #print("INPUT IN FORWARD IS",input)
        output = self.main(input.cuda())
        #print("OUTPUT (self.main) IN FORWARD IS: " ,output)
        #print("OUTPUT .view(-1))", output.view(-1))
        return output.view(-1).cuda()

netD = D().cuda()
netD.apply(weights_init).cuda()

In [ ]:
criterion = nn.BCELoss().cuda()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))
epochBound = 30

for epoch in range(epochBound):

    for i, data in enumerate(data_loader, 0):
        
        netD.zero_grad()
        
        real, _ = data
        input = Variable(real)
        #print("INPUT IS:", input)
        target = Variable(torch.ones(input.size()[0]).cuda()).cuda()
        #print("TAGET IS:", target)
        output = netD(input).cuda()
        #print("OUTPUT IS:", output)
        #target = Variable(torch.ones(output.size()[0]))
        errD_real = criterion(output, target).cuda()
        
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1).cuda())
        fake = netG(noise).cuda()
        target = Variable(torch.zeros(input.size()[0]).cuda())
        output = netD(fake.detach()).cuda()
        errD_fake = criterion(output, target)
        
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]).cuda()).cuda()
        output = netD(fake).cuda()
        #arget = Variable(torch.ones(output.size()[0]))
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, epochBound, i, len(data_loader), errD.data[0], errG.data[0]))
        if i % 50 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./GANitR1", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./GANitR1", epoch), normalize = True)

In [ ]:
import os
os.getcwd()